In [2]:
#!/usr/bin python2.7

from astropy.io       import fits
from astropy.table    import Table
from astropy.time     import Time
from datetime         import date
from datetime         import datetime
import ephem
import fileinput
import getpass
import math
import matplotlib.pyplot                   as plt
import matplotlib                          as mpl
import numpy                               as np
from numpy            import linalg        as LA
import os
import pyfits
import scipy.optimize                      as optimization
from scipy            import integrate
from scipy            import interpolate
from scipy.optimize   import curve_fit
import subprocess
from subprocess       import Popen, PIPE
import urllib2
from work_module      import calculate
from work_module      import detector
from work_module      import readfile
from work_module      import writefile
calc = calculate()
det = detector()
rf = readfile()
wf = writefile()

In [7]:
detectors = np.array(['n0', 'n1', 'n2', 'n3', 'n4', 'n5', 'n6', 'n7', 'n8', 'n9', 'na', 'nb'])
#detectors = np.array(['n0', 'n1', 'n2', 'n3', 'n4', 'n5'])

day = 111209
burst_ra = 299.59
burst_dec = 35.202

start_time = 7
end_time = 8.5

v_line = 4.90648687098


burst_occ = calc.src_occultation(day, burst_ra, burst_dec)[0]

sun = calc.sun_pos(day)[0]
burst_pos = np.array([np.cos(burst_ra*(2.*math.pi)/360.)*np.cos(burst_dec*(2.*math.pi)/360.), np.sin(burst_ra*(2.*math.pi)/360.)*np.cos(burst_dec*(2.*math.pi)/360.), np.sin(burst_dec*(2.*math.pi)/360.)], float) #unit-vector pointing to the burst location
scalar_product = sun[0]*burst_pos[0] + sun[1]*burst_pos[1] + sun[2]*burst_pos[2]
sun_ang = np.arccos(scalar_product)
sun_ang = (sun_ang)*360./(2.*math.pi) #convert to degrees
sun_ang = np.array(sun_ang)

#print sun_ang

sat_time = rf.poshist(day)[0]

plot_time_sat_date = calc.met_to_date(sat_time)[0]
plot_time_sat = (plot_time_sat_date - calc.day_to_met(day)[1])*24#Time of day in hours

mpl.rcParams['axes.color_cycle'] = ['b', 'g', 'r', 'c', 'm', 'y', 'k']

index = np.where(plot_time_sat > v_line)[0][0]

for i in range(0, len(detectors)):
    #print detectors[i]
    det_data = calc.burst_ang(detectors[i], day, burst_ra, burst_dec)
    det_ang = det_data[0]
    
    print detectors[i], ': ', det_ang[index]
    #sat_time = det_data[1]
    #det_ang[np.where(det_ang > 90.)] = 0.
    
    plt.plot(plot_time_sat, det_ang, label = detectors[i])
    
plt.axvline(x = v_line, ymin = 0., ymax = 1., linewidth=2, color = 'grey')

plt.plot(plot_time_sat, burst_occ, label = 'Earth-Occultation')    
plt.plot(plot_time_sat, sun_ang, 'b-.',label = 'Sun')    
plt.legend(fontsize=15)
plt.tick_params(axis='both', which='major', labelsize=15)

plt.xlabel('Daytime in 24h', fontsize=20)
plt.ylabel('Angle in degrees', fontsize=20)
plt.title('Winkelverhaeltnisse zwischen dem IceCube Neutrino-Event und den Fermi-Detektoren', fontsize=30)

plt.xlim([start_time, end_time])
plt.ylim([-10, 200])

plt.show()

n0 :  115.133504556
n1 :  119.031062413
n2 :  124.675283035
n3 :  66.1153352257
n4 :  20.926194678
n5 :  72.685437373
n6 :  98.992056404
n7 :  86.8169970544
n8 :  56.9572646951
n9 :  146.906371897
na :  158.180725296
nb :  106.909926314


In [8]:
for detector in detectors:
    earth_ang = calc.earth_ang(detector, day)[0]
    earth_occ = calc.earth_occ_eff(earth_ang, echan = 2)
    
    #print detector, ': ', earth_ang[index]
    print detector, ': ', earth_occ[index]

n0 :  0.118392654881
n1 :  0.0923046074742
n2 :  0.139870238475
n3 :  0.238835357391
n4 :  0.387995140717
n5 :  0.215111424304
n6 :  0.228737648581
n7 :  0.323634152961
n8 :  0.481918426544
n9 :  0.174170109545
na :  0.227922800418
nb :  0.399834775927


In [9]:
print calc.earth_occ_eff(0, echan = 2)

0.539303607418
